In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import sys, os, time, json
from pathlib import Path
import pickle as pkl

sys.path.append('../PreProcessing/')
sys.path.append('../Lib/')
sys.path.append('../Analyses/')

import analyses_table as AT
import stem_branch_analyses as SBA
from importlib import reload  # Python 3.4+ only.


In [2]:
oakPaths = AT.getOakPaths()

In [ ]:
AT = reload(AT)
SBA = reload(SBA)

task = 'T3g'

animals = ['Li','Ne','Cl']
overwrite=False
doPlots = False

All_UniRes = pd.DataFrame()
All_DecRes = pd.DataFrame()
All_PopDecRes = pd.DataFrame()
for animal in animals:
    
    fn = oakPaths['Root'] / (animal+'_SessionPaths.pkl')
    with fn.open(mode="rb") as f:
        sePaths = pkl.load(f)

    CT_Path =  oakPaths['Clustered'] / animal / (animal+'_ClusteringSummary.json')
    CueDesDat_Fn = oakPaths['Analyses'] / animal / ('{}_{}_CueDesSegRes.csv').format(animal,task)
    CueDesDecDat_Fn = oakPaths['Analyses'] / animal / ('{}_{}_CueDesSegDecRes.csv').format(animal,task)
    CueDesPopDecDat_Fn = oakPaths['Analyses'] / animal / ('{}_{}_CueDesSegDecPopRes.csv').format(animal,task)
    with CT_Path.open() as f:
        CT = json.load(f)
    
    SessionList = []
    cnt=0
    uniRes = pd.DataFrame()
    decRes = pd.DataFrame()
    popDecRes = pd.DataFrame()
    for se,v in CT['Sessions'].items():
        if v:
            temp = se.split('_')
            if temp[1]==task:
                try:
                    print('Analysing Session {}'.format(se))
                    SessionList.append(se)
                    x, _, y, z = SBA.main(sePaths[se],overwrite=overwrite,doPlots=doPlots)
                    if isinstance(x,pd.DataFrame):
                        uniRes = pd.concat((uniRes,x))
                        decRes = pd.concat((decRes,y))
                        popDecRes = pd.concat((popDecRes,z))
                except:
                    print("Error processing session: {}".format(se))
                    print ("Error", sys.exc_info()[0],sys.exc_info()[1],sys.exc_info()[2].tb_lineno)
                    
        uniRes.to_csv(CueDesDat_Fn)
        decRes.to_csv(CueDesDecDat_Fn)
        popDecRes.to_csv(CueDesPopDecDat_Fn)
    
    uniRes['an'] = animal
    decRes['an'] = animal
    popDecRes['an'] = animal
    
    All_UniRes = pd.concat((All_UniRes,uniRes))
    All_DecRes = pd.concat((All_DecRes,decRes))
    All_PopDecRes = pd.concat((All_PopDecRes,popDecRes))


Analysing Session Li_T3g_052818
Loading Beh Tracking Data
Complete.
.......................................................................................................................
Decoding Correct Model Completed. Time  = 166.58s 

.......................................................................................................................
Decoding Balanced  Model Completed. Time  = 108.76s 

.......................................................................................................................
Decoding Incorrect Model Completed. Time  = 73.19s 

Analysing Session Li_T3g_052918
Loading Beh Tracking Data
Complete.
.............................................................................
Decoding Correct Model Completed. Time  = 94.00s 

.............................................................................
Decoding Balanced  Model Completed. Time  = 100.08s 

.............................................................................
Decodi

In [9]:
se

'Li_T3i_061218'